# Mouvements de joystick

## Marquers

In [ ]:
%load_ext autoreload 
%autoreload 2

In [1]:
import pickle
import numpy as np
import pandas as pd
import time

from scipy import signal
from collections import Counter

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from scipy.signal import periodogram

from os.path import exists
import pickle

varr = {}
varr['main_path'] = "/home/oem2/Documents/9_Motor_classification_2018-22/Coding_version3_python_FINAL"  
varr['main_path1'] = "%s/a_data_standardization" % (varr['main_path'])
varr['main_path2'] = "%s/b_data_preprocessing" % (varr['main_path'])
varr['main_path3'] = "%s/c_calculate_metrics" % (varr['main_path'])

import sys
sys.path.insert(1, '%s' % (varr['main_path']))

# Personal python functions
from c_calculate_metrics.put_timeseries_trialdata_into_pandas import *

# from subfunctions.make_a_properlist import *
from subfunctions.numderiv import *
from subfunctions.freqresp_functions import get_freqresp_mag_phase, select_fc, select_filter
from subfunctions.freq_from_sig_timecounting import *
from subfunctions.freq_from_sig_freqresp import *
from subfunctions.my_dropna_python import *
from subfunctions.scale_feature_data import *
from subfunctions.normal_distribution_feature_data import *
from subfunctions.findall import *
from subfunctions.explode_without_colnames2 import *
#from subfunctions.scikit_functions_binaryclass import *
#from subfunctions.scikit_functions import *

from subfunctions.tsig_2_discrete_wavelet_transform import *
from subfunctions.tsig_2_spectrogram import *
from subfunctions.tsig_2_continuous_wavelet_transform import *

df_timeseries_exp = put_timeseries_trialdata_into_pandas(varr)

In [2]:
print('shape of df_timeseries_exp[rot] : ', df_timeseries_exp['rot'].shape)
df_timeseries_exp['rot'].tail()

shape of df_timeseries_exp[rot] :  (86006, 19)


,subject,tr,ss,ax,dp,time,res_type,SIGCOM_ax0,SIGCOM_ax1,SIGCOM_ax2,SIG_ax0,SIG_ax1,SIG_ax2,JOY_ax0,JOY_ax1,JOY_ax2,NOISE_ax0,NOISE_ax1,NOISE_ax2
86001,17.0,20.0,-1.0,0.0,163.0,16.300000,7.0,-1.453208,0.804159,-4.699437,-1.514817,0.803347,-4.719625,0.301392,-0.082375,-4.927108,0.850650,0.0,-4.927108
86002,17.0,20.0,-1.0,0.0,164.0,16.400000,7.0,-1.391906,0.804159,-4.699425,-1.461697,0.803352,-4.719593,0.333524,-0.082369,-4.917930,0.902199,0.0,-4.917930
86003,17.0,20.0,-1.0,0.0,165.0,16.500001,7.0,-1.313130,0.804159,-4.699425,-1.382446,0.807940,-4.719292,0.356304,-0.086946,-4.908213,0.953649,0.0,-4.908213
86004,17.0,20.0,-1.0,0.0,166.0,16.599999,7.0,-1.221655,0.804159,-4.699425,-1.284506,0.807370,-4.719829,0.362870,-0.089799,-4.897957,1.004994,0.0,-4.897957
86005,17.0,20.0,-1.0,0.0,167.0,16.700000,7.0,-1.125844,0.804159,-4.699425,-1.179229,0.805081,-4.719719,0.364322,-0.088796,-4.887165,1.056230,0.0,-4.887165


In [3]:
print('shape of df_timeseries_exp[trans] : ', df_timeseries_exp['trans'].shape)
df_timeseries_exp['trans'].tail()

shape of df_timeseries_exp[trans] :  (43754, 19)


,subject,tr,ss,ax,dp,time,res_type,SIGCOM_ax0,SIGCOM_ax1,SIGCOM_ax2,SIG_ax0,SIG_ax1,SIG_ax2,JOY_ax0,JOY_ax1,JOY_ax2,NOISE_ax0,NOISE_ax1,NOISE_ax2
43749,13.0,16.0,1.0,1.0,197.0,19.7,7.0,0.0,69.3868,51.1975,-1.3760,68.5344,31.0360,-0.0412,-0.0293,0.0,3.75,-0.0,0.0
43750,13.0,16.0,1.0,1.0,198.0,19.8,7.0,0.0,69.7618,51.4284,-1.3559,68.9152,31.1329,-0.0412,-0.0294,0.0,3.75,-0.0,0.0
43751,13.0,16.0,1.0,1.0,199.0,19.9,7.0,0.0,70.1368,51.6561,-1.3360,69.2979,31.2316,-0.0412,-0.0272,0.0,3.75,-0.0,0.0
43752,13.0,16.0,1.0,1.0,200.0,20.0,7.0,0.0,70.5118,51.7815,-1.3128,69.6821,31.3353,-0.0408,-0.0273,0.0,3.75,-0.0,0.0
43753,13.0,16.0,1.0,1.0,201.0,20.1,7.0,0.0,70.7139,51.7700,-1.2916,70.0518,31.4619,-0.0408,-0.0273,0.0,0.00,0.0,0.0


In [4]:
def get_df(ax_val, ss_val):
    
    if ax_val == 'all' and ss_val == 'all':
        # All the data
        df_timeseries_exp[exp].head()
        df = df_timeseries_exp[exp]
    elif ax_val != 'all' and ss_val == 'all':
        # Prediction for each axis
        if ax_val == 'ax0':
            ax_val_n = 0
        elif ax_val == 'ax1':
            ax_val_n = 1
        elif ax_val == 'ax2':
            ax_val_n = 2
        df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ax == ax_val_n)]
    elif ax_val == 'all' and ss_val != 'all':
        # Prediction per sup/sub
        if ss_val == 'sup':  # sup
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss > 0)]
        elif ss_val == 'sub':  # sub
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss < 0)]
    elif ax_val != 'all' and ss_val != 'all':
        # Prediction per axis and sup/sub
        if ax_val == 'ax0':
            ax_val_n = 0
        elif ax_val == 'ax1':
            ax_val_n = 1
        elif ax_val == 'ax2':
            ax_val_n = 2
        
        if ss_val == 'sup':  # sup
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss > 0) & (df_timeseries_exp[exp].ax == ax_val_n)]
        elif ss_val == 'sub':  # sub
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss < 0) & (df_timeseries_exp[exp].ax == ax_val_n)]

    print('Confirmation : exp=', exp, ', ax_val=', ax_val, ', ss_val=', ss_val)
    
    # Reset the index
    df.reset_index(drop=True, inplace=True)
    
    return df

In [5]:
# Get the joystick stimulus axis data and put it in a pandas column
def indexit(row):
    joy_mat = [row.JOY_ax0, row.JOY_ax1, row.JOY_ax2]
    return joy_mat[int(row.ax)]

In [6]:
def create_labels_and_initial_feature(df):
    # IC = 1
    # EC = 2, 4, 5
    # NC = 3, 6, 7
    # NR = 9
    # (IC) - sham (do not use) = 8
    # (NC) - sham (do not use) = 10

    # Just to confirm, what are the unique values of res_type
    df.res_type.value_counts(ascending=True)

    # Construction of SD_label : How do we define disorientation?

    # Way 0 : lenient
    # 0 = If participates got the result CORRECT for the trial, they were NOT disoriented. (IC, EC)
    # 1 = If participates got the result WRONG or did not respond, they were disoriented. (NC, NR)

    idx_NDS = df.index[(df.res_type == 1) | (df.res_type == 2) | (df.res_type == 4) | (df.res_type == 5)].to_list()
    idx_DS = df.index[(df.res_type == 3) | (df.res_type == 6) | (df.res_type == 7) | (df.res_type == 9)].to_list()
    df.lenient = ''  # define a new column , rows 8 and 10 will be NaN, need to do dropna for rows
    df.loc[idx_NDS, 'lenient'] = 0
    df.loc[idx_DS, 'lenient'] = 1

    # -------------------------------------

    # Way 1 : strict simple
    # 0 = If participants got the result initially CORRECT, they were NOT disoriented. (IC)
    # 1 = If participants were WRONG at any point, they were disoriented. (EC, NC, NR)

    idx_NDS = df.index[(df.res_type == 1)].to_list()
    idx_DS = df.index[(df.res_type == 2) | (df.res_type == 4) | (df.res_type == 5) | (df.res_type == 3) | (df.res_type == 6) | (df.res_type == 7) | (df.res_type == 9)].to_list()
    df.strict = ''  # define a new column
    df.loc[idx_NDS, 'strict'] = 0
    df.loc[idx_DS, 'strict'] = 1

    # Way 2 : st_complex
    # 0 = If participants got the result initially CORRECT, they were NOT disoriented. (IC)
    # 1 = If participants got the result eventually CORRECT, they were MILDLY disoriented. (EC)
    # 2 = If participants were WRONG for the trial, they were disoriented. (NC, NR)

    idx_NDS = df.index[(df.res_type == 1)].to_list()
    idx_MDS = df.index[(df.res_type == 2) | (df.res_type == 4) | (df.res_type == 5)].to_list()
    idx_DS = df.index[(df.res_type == 3) | (df.res_type == 6) | (df.res_type == 7) | (df.res_type == 9)].to_list()
    df.st_complex = ''  # define a new column
    df.loc[idx_NDS, 'st_complex'] = 0
    df.loc[idx_MDS, 'st_complex'] = 1
    df.loc[idx_DS, 'st_complex'] = 2

    # -------------------------------------

    # Create features :  (1) position
    df['joy_stim'] = df.apply(indexit, axis='columns')  # fill in joy_stim
    
    # -------------------------------------

    # Make DataFrame for trial start-stop index
    # Cut the data up per trial across subjects
    tr_vec = df.tr.to_numpy()

    st = [0]
    ender = []
    for i in range(len(tr_vec)-1):
        if tr_vec[i] != tr_vec[i+1]:
            st = st + [i+1]
            ender = ender + [i]
    ender = ender + [len(tr_vec)-1]

    # See start-stop index clearly
    e0 = np.reshape(st, (len(st),1))
    e1 = np.reshape(ender, (len(st),1))
    data = np.ravel(e0), np.ravel(e1)
    data = np.transpose(data)
    columns = ['stind', 'endind']
    temp = pd.DataFrame(data=data, columns=columns)

    # -------------------------------------

    # Find the longest trial signal in df_rot['joy_stim']
    temp['diff'] = temp.endind - temp.stind
    temp['timediff'] = [df.time.iloc[temp.endind[i]] - df.time.iloc[temp.stind[i]] for i in range(len(temp.endind))]
    outmin = temp['diff'].min()
    outmax = temp['diff'].max()

    tomin = temp['timediff'][(temp['diff'] == outmin)]
    tomax = temp['timediff'][(temp['diff'] == outmax)]
    # print('outmin : ', outmin, 't :', tomin)
    # print('outmax : ', outmax, 't :', tomax)

    # -------------------------------------

    # Interpolate : make each trial the same number of data points
    from scipy.interpolate import interp1d
    feat0 = []
    t_feat0 = []
    y1_feat0 = []
    y2_feat0 = []
    y3_feat0 = []
    for i in range(len(temp.stind)):
        
        # X
        sSIG = df['joy_stim'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        t_sSIG = df['time'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()

        # labels
        y1 = df['lenient'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        y2 = df['strict'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        y3 = df['st_complex'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        
        # Check if trial data is less than the maximum length
        if len(df['joy_stim'][temp.stind.iloc[i]:temp.endind.iloc[i]]) < outmax:
            
            # The trial length is different so interpolate the time-series to make them the same length signal 
            x = np.linspace(sSIG[0], len(sSIG), num=len(sSIG), endpoint=True)
            xnew = np.linspace(sSIG[0], len(sSIG), num=outmax, endpoint=True)

            # joystick on stim
            f = interp1d(x, sSIG)
            sSIGl = f(xnew)

            # time
            f = interp1d(x, t_sSIG)
            t_sSIGl = f(xnew)

            # y1
            f = interp1d(x, y1)
            y1_sSIGl = f(xnew)

            # y2
            f = interp1d(x, y2)
            y2_sSIGl = f(xnew)

            # y3
            f = interp1d(x, y3)
            y3_sSIGl = f(xnew)

            # python : you can not create a matrix in real-time in pandas
            # you only assign the full matrix at the end
            # (0) position
            feat0 = feat0 + [sSIGl]
            t_feat0 = t_feat0 + [t_sSIGl]
            y1_feat0 = y1_feat0 + [np.ravel(y1_sSIGl)]
            y2_feat0 = y2_feat0 + [np.ravel(y2_sSIGl)]
            y3_feat0 = y3_feat0 + [np.ravel(y3_sSIGl)]
            
            del x, f, sSIGl, t_sSIGl, y1_sSIGl, y2_sSIGl, y3_sSIGl
        else:
            feat0 = feat0 + [sSIG]
            t_feat0 = t_feat0 + [t_sSIG]
            y1_feat0 = y1_feat0 + [np.ravel(y1)]
            y2_feat0 = y2_feat0 + [np.ravel(y2)]
            y3_feat0 = y3_feat0 + [np.ravel(y3)]

    # Clean up
    del df
    # -------------------------------------
    
    return feat0, t_feat0, y1_feat0, y2_feat0, y3_feat0

In [7]:
def feature_creation_preprocessing(feat0, t_feat0):

    start = time.time()
    
    # ----------------
    # Make your features
    # ----------------
    df_feat = pd.DataFrame()
    
    n = 4   # filter order
    fs = 250 # data sampling frequency (Hz)
    fcc = 10  # Cut-off frequency of the filter
    w = fcc / (fs / 2) # Normalize the frequency
    b, a = signal.butter(n, w, 'low')  # 3rd order
    
    scales = np.arange(1, 128)
    
    # Need to find when one trial starts and end - take derivative from start-stop periods
    for i in range(len(feat0)):

        if i == 0:
            plotORnot = 1
        else:
            plotORnot = 0
        
        # ----------------------------
        # (0) position - causale ordre
        col0 = scale_feature_data(feat0[i], plotORnot)
        
        # (1) velocity - causale ordre
        vel = numderiv(feat0[i], t_feat0[i])
        col1 = scale_feature_data(vel, plotORnot)

        # (2) acceleration - causale ordre
        acc = numderiv(vel, t_feat0[i])
        filtacc = signal.filtfilt(b, a, acc) # the signal is noisy
        col2 = scale_feature_data(filtacc, plotORnot)
        # ----------------------------
        
        # ----------------------------
        # (4) position - non-causale ordre
        col3 = normal_distribution_feature_data(col0, plotORnot)
        
        # (5) velocity - non-causale ordre
        col4 = normal_distribution_feature_data(col1, plotORnot)
        
        # (6) acceleration - non-causale ordre
        col5 = normal_distribution_feature_data(col2, plotORnot)
        # ----------------------------
        
        
        
        # ----------------------------
        # Frequence marquers : sublevels of frequency pattern
        # ----------------------------
        # (7-22) une transformation de fréquence (ondelettes)
        coeff = tsig_2_discrete_wavelet_transform(col0, waveletname='sym5', level=5, plotORnot=0)
        cols6 = pd.DataFrame(coeff).T
        
        coeff = tsig_2_discrete_wavelet_transform(col1, waveletname='sym5', level=5, plotORnot=0)
        cols7 = pd.DataFrame(coeff).T
        
        coeff = tsig_2_discrete_wavelet_transform(col2, waveletname='sym5', level=5, plotORnot=0)
        cols8 = pd.DataFrame(coeff).T
        # ----------------------------
        
        
        # ----------------------------
        # Hybrid marquers : temporalle et frequence information
        # ----------------------------
        # (8) spectrogram flatten - periodogram (fft)
        col9 = tsig_2_spectrogram(col0, fs=10, nfft=20, noverlap=0, plotORnot=0)
        
        col10 = tsig_2_spectrogram(col1, fs=10, nfft=20, noverlap=0, plotORnot=0)
        
        col11 = tsig_2_spectrogram(col2, fs=10, nfft=20, noverlap=0, plotORnot=0)
        # ----------------------------
        
        # ----------------------------
        # (9) continuous wavelet transform flatten
        # continuous_wavelets = ['mexh', 'morl', 'cgau5', 'gaus5']
        col12 = tsig_2_continuous_wavelet_transform(t_feat0[i], col0, scales, waveletname='mexh', plotORnot=0)
        
        col13 = tsig_2_continuous_wavelet_transform(t_feat0[i], col1, scales, waveletname='mexh', plotORnot=0)
        
        col14 = tsig_2_continuous_wavelet_transform(t_feat0[i], col2, scales, waveletname='mexh', plotORnot=0)
        # ----------------------------
        
        
        # Peut-être faire des non-causale frequence marquers pour des mieux frequence et hybrid 
        
        tr = pd.Series(i*np.ones(len(col0)))
        c0 = pd.Series(col0)
        c1 = pd.Series(col1)
        c2 = pd.Series(col2)
        c3 = pd.Series(col3)
        c4 = pd.Series(col4)
        c5 = pd.Series(col5)
        cs6 = cols6
        cs7 = cols7
        cs8 = cols8
        c9 = pd.Series(col9)
        c10 = pd.Series(col10)
        c11 = pd.Series(col11)
        
        c12 = pd.Series(col12)
        c13 = pd.Series(col13)
        c14 = pd.Series(col14)
        
        # Clean up
        del vel, acc, filtacc, coeff
        del col0, col1, col2, col3, col4, col5, cols6, cols7, cols8, col9, col10, col11, col12, col13, col14
        
        temp = pd.concat([tr, c0, c1, c2, c3, c4, c5, cs6, cs7, cs8, c9, c10, c11, c12, c13, c14], axis=1)
        
        # Clean up
        del tr, c0, c1, c2, c3, c4, c5, cs6, cs7, cs8, c9, c10, c11, c12, c13, c14
        
        df_feat = pd.concat([df_feat, temp], axis=0)
        
        del temp
        
    # ----------------

    end = time.time()
    print('Elasped time for feature processing : ', end - start)

    return df_feat

In [8]:
def pandas_rename_columns(df, col_list):
    
    way = 1
    
    if way == 0:
        # Façon difficile
        # Rename columns
        onames = df.columns.to_numpy()
        dictout = {}
        for nf in range(len(col_list)):
            dictout[onames[nf]] = '%s' % (col_list[nf])
            
        # Determinez quels columns de df repeter
        uq = Counter(onames).most_common()
        d = {}
        for i in range(len(uq)):
            temp = []
            for ind, val in enumerate(onames):
                if uq[i][0] == val:
                    temp.append(col_list[ind])
            d[i] = temp    
        
        # if the column name is a key of d pop the names in the list, else return the column name
        df.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
    
    elif way == 1:
        # Façon facile
        df.columns = col_list
    
    return df

In [9]:
def pad_data_2makeclasses_equivalent(df_feat):
    # Remove nan value per row
    # df_test_noNan = df_feat.dropna(axis=0)
    # OR
    df_test_noNan = my_dropna_python(df_feat)

    # ----------------

    # Confirm that there are no nan values
    out = df_test_noNan.isnull().values.any()
    print('Are there nan valeus in the data : ', out)

    # ----------------

    # Check class balance
    needed_samps_class, counted_value, count_index, st, endd = count_classes(df_test_noNan)

    # ----------------

    print('shape of dataframe before padding : ', df_test_noNan.shape)

    # ----------------
    # Pad the DataFrame
    n_classes = len(count_index)
    n_samples = len(st)

    df_2add_on = pd.DataFrame()
    
    # Le derniere sample dans df_test_noNan
    df_coupe_proche = df_test_noNan.iloc[st[-1]:endd[-1], :]

    for i in range(n_classes):
        #print('i : ', i)
        # Pad short length classes
        for j in range(needed_samps_class[i]):
            #print('j : ', j) 
            flag = 0
            while flag == 0:
                permvec = np.random.permutation(n_samples)
                index = permvec[0]  #random choosen index
                
                # look for each class : on veut le classe être le meme
                if i == int(df_test_noNan.y.iloc[st[index]]):
                    #print('Class match was found : i = ', i, ', data index = ', int(df_test_noNan.y_scalar.iloc[index]), ', index = ', index)
                    
                    # Append the data with padded data entry
                    df_coupe = df_test_noNan.iloc[st[index]:endd[index], :]
                    
                    # Le derriere sample ne sont pas le meme que le sample actuelle
                    if int(df_coupe.iloc[0,0] - df_coupe_proche.iloc[0,0]) != 0:
                        df_coupe_proche = df_coupe
                        df_2add_on = pd.concat([df_2add_on, df_coupe], axis=0)
                        flag = 1 # to brake while
                        
    # ----------------

    # DataFrame a besoin les noms de columns d'avoir le meme noms que df_test_noNan
    df_2add_on = df_2add_on.reset_index(drop=True)  # reset index : delete the old index column

    col_list = df_test_noNan.columns
    df_2add_on = pandas_rename_columns(df_2add_on, col_list)
    df_2add_on

    print('shape of dataframe to add to original dataframe: ', df_2add_on.shape)

    # ----------------

    # want to arrange the dataframe with respect to rows (stack on top of the other): so axis=0 
    # OR think of it as the rows of the df change so you put axis=0 for rows
    df_test2 = pd.concat([df_test_noNan, df_2add_on], axis=0)
    df_test2 = df_test2.reset_index(drop=True)  # reset index : delete the old index column

    print('shape of padded dataframe (original + toadd) : ', df_test2.shape)

    del df_test_noNan, df_2add_on

    # ----------------

    # Final check of class balance
    needed_samps_class, counted_value, count_index, st, endd = count_classes(df_test2)

    # ----------------
    
    # Enlevez des columns unnecessaires : num, y
    # df_test2 = df_test2.drop(['num', 'y'], axis=1)  # 1 is the axis number (0 for rows and 1 for columns)

    # ----------------

    # Rename each feature column a number, and the label column with y 
    # col_list = list(map(str, np.arange(len(df_test2.columns) - 1)))
    # col_list.append('y')
    # df_test2 = pandas_rename_columns(df_test2, col_list)
    
    return df_test2

In [10]:
def detect_nonconsecutive_values_debut_fin_pt(vec):
  
    st = [0]
    endd = []
    
    for i in range(len(vec)-1):
        if vec[i] != vec[i+1]:
            st.append(i+1)
            endd.append(i)
    
    endd.append(len(vec)-1)
    
    return st, endd

In [11]:
def count_classes(df_test_noNan):

    # Get start and end index values for each sample
    num = list(map(int, df_test_noNan.num.to_numpy()))
    st, endd = detect_nonconsecutive_values_debut_fin_pt(num)

    # ----------------

    yy = list(map(int, df_test_noNan.y.to_numpy()))
    y_short = []
    for i in range(len(st)):
        y_short.append(yy[st[i]:st[i]+1][0])

    # ----------------

    liste = Counter(y_short).most_common()
    count_index, counted_value = list(map(list, zip(*liste)))

    print('Before sorting counted_value : ', counted_value)
    print('Before sorting count_index : ', count_index)

    # ----------------

    # Sort counted_value by count_index; in ascending order
    sind = np.argsort(count_index)
    count_index = [count_index[i] for i in sind]
    counted_value = [counted_value[i] for i in sind]
    print('After sorting counted_value : ', counted_value)
    print('After sorting count_index : ', count_index)

    # ----------------

    # Determine how much to pad each class label
    needed_samps_class = np.max(counted_value) - counted_value
    print('needed_samps_class : ', needed_samps_class)

    # ----------------
    
    return needed_samps_class, counted_value, count_index, st, endd

In [12]:
def save_dat_pickle(outSIG, file_name="outSIG.pkl"):
    # Save data matrices to file
    open_file = open(file_name, "wb")
    pickle.dump(outSIG, open_file)
    open_file.close()

In [13]:
def load_dat_pickle(file_name="outSIG.pkl"):
    open_file = open(file_name, "rb")
    dataout = pickle.load(open_file)
    open_file.close()
    return dataout

## Verifier des marquers : Plot des marquers

In [ ]:
import seaborn as sns

plt.figure(figsize=(14,6))  # width, height of the figure
sns.set(style="whitegrid")

sns.lineplot(data=df_feat.pos, label="num")
sns.lineplot(data=df_feat.vel, label="num", color='red')
sns.lineplot(data=df_feat.acc, label="num", color='green')
plt.title("position, velocity, acceleration")
plt.xlabel("sample")

In [ ]:
plt.figure(figsize=(14,6))  # width, height of the figure
sns.set(style="whitegrid")

sns.lineplot(data=df_feat.pos_sl0, label="position")
sns.lineplot(data=df_feat.vel_sl0, label="velocity", color='red')
sns.lineplot(data=df_feat.acc_sl0, label="acceleration", color='green')
plt.title("wavelet sublevel : position, velocity, accleration")
plt.xlabel("sample")

In [ ]:
plt.figure(figsize=(14,6))  # width, height of the figure
sns.set(style="whitegrid")

sns.lineplot(data=df_feat.pos_spec, label="pos_spec")
sns.lineplot(data=df_feat.vel_spec, label="vel_spec", color='red')
sns.lineplot(data=df_feat.acc_spec, label="acc_spec", color='green')
plt.title("Spectrogram")
plt.xlabel("sample")

In [ ]:
plt.figure(figsize=(14,6))  # width, height of the figure
sns.set(style="whitegrid")

sns.lineplot(data=df_feat.pos_cwt, label="pos_cwt")
sns.lineplot(data=df_feat.vel_cwt, label="vel_cwt", color='red')
sns.lineplot(data=df_feat.acc_cwt, label="acc_cwt", color='green')
plt.title("Continuous Wavelet Transform")
plt.xlabel("sample")

## Classification

In [14]:
def sort_dict_by_value(d, reverse = False):
    return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))



def permutation_importance_tensorflow(model, X_test, Y_test):

    Y_test_1D = [Y_test[i,0:1] for i in range(Y_test.shape[0])]

    # First, a baseline metric, defined by scoring,
    # Obtenez mean absolute error
    y_hat_test = model.predict(X_test, verbose=0)
    baseline_mae = np.mean(np.abs(y_hat_test - Y_test_1D))

    vals = {}
    # Shuffle each feature columns at a time
    for featcol in range(X_test.shape[2]):

        # Define a modifiable temporary variable
        temp = X_test

        # select a column
        feat_slice = temp[:,:,featcol]

        # Must flatten the matrix because np.random.permutation or 
        # np.random.shuffle don't work
        t = feat_slice.flatten()
        t_shuf = np.random.permutation(t)
        feat_slice =  np.reshape(t_shuf, (feat_slice.shape))

        # put feat_slice back into temp
        temp[:,:,featcol] = feat_slice

        y_hat_test = model.predict(temp, verbose=0)
        mae_per_col = np.mean(np.abs(y_hat_test - Y_test_1D))
        vals[featcol] = mae_per_col

    # Sort the columns from largest to smallest mae
    laquelle = sort_dict_by_value(vals, reverse = True)
    
    # Determinez le nombres des columns qui sont plus grande que le baseline_mae
    # C'est des marqueurs qui sont importants
    feat = list(laquelle.keys())
    cnt = [1 for i in range(len(feat)) if feat[i] > baseline_mae]
    cnt = np.sum(cnt)
    
    allout = list(laquelle.items())
    nout = [allout[i] for i in range(cnt)]
    marquers_important = dict(nout)
    
    return marquers_important

In [15]:
def classify(df, ynum):
    
    # Loop over each model to test
    for wm in [1, 0]:
        res_permod = []
        
        for fea in range(4):
            print('fea : ', fea)
            
            # ----------------
            # Order of which features to use in a model
            if fea == 0:
                # 1) All features
                X_cols = list(np.arange(1, df_test2.shape[1]-1, 1))
                y_cols = [df_test2.shape[1]-1]
            elif fea == 1:
                # 2) first 3 from permutation_importance
                X_cols = [list(marquers_important.items())[oo][0] for oo in range(3)]
                y_cols = [df_test2.shape[1]-1]
            elif fea == 2:
                # 3) first 2 from permutation_importance
                X_cols = [list(marquers_important.items())[oo][0] for oo in range(2)]
                y_cols = [df_test2.shape[1]-1]
            elif fea == 3:
                # 4) first feature from permutation_importance
                X_cols = [list(marquers_important.items())[0][0]]
                y_cols = [df_test2.shape[1]-1] 
            
            if wm == 0:
                # Sequential : Support Vector Machine (NuSVC)
                m_name = 'NuSVC'
                X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2)
                
                # Determine if classes are binary or multiclass:
                class_len = len(np.unique(Y_train_1D))
                if class_len <= 2:
                    model, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test = binary_svm_NuSVC_batch(X_train, X_test, Y_train_1D, Y_test_1D, batch_size)
                elif class_len > 2:
                    model, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test = multiclass_svm_NuSVC_batch(X_train, X_test, Y_train_1D, Y_test_1D, batch_size)
                
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    from sklearn.inspection import permutation_importance
                    r = permutation_importance(model, X_test, Y_test_1D, n_repeats=10, random_state=0, scoring='accuracy')
                    vals = dict(zip(np.arange(len(r.importances_mean)), r.importances_mean))
                    marquers_important = sort_dict_by_value(vals, reverse = True) # Sort the columns from largest to smallest mae
                # ----------------
                
            elif wm == 1:
                # Sequential : LSTM - changes within a window of points
                m_name = 'LSTM'
                model, dict_out, X_test, Y_test = run_LSTM(df, X_cols, y_cols, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = [dict_out['delay_train'], dict_out['delay_test']]    
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                
            elif wm == 2:
                # Sequential : Transformer - changes between windows of points
                m_name = 'Trans'
                dict_out, X_test, Y_test = run_Transformer(df, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                
            elif wm == 3:
                # Spatial (find global trends in the feature) : RandomForest - partitioned subspace
                m_name = 'RF'
                X_train, X_test, Y_train_1D, Y_test_1D = preprocess_df_2_XYtraintest(df, X_cols, y_cols)
                
                # Determine if classes are binary or multiclass:
                class_len = len(np.unique(Y_train_1D))
                if class_len <= 2:
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_RandomForest(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif class_len > 2:
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_RandomForest_1Dinput(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
                
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(value_pack_train, q=2)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(value_pack_test, q=3)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    col_noms = ['%s' % (i) for i in range(X_test.shape[1])]
                    marquers_important = pipeline_permutation_importance(model, X_test, Y_test_1D, col_noms)
                # ----------------
                
            elif wm == 4:
                # Spatial (find global trends in the feature) : CNN
                m_name = 'CNN'
                dict_out, X_test, Y_test, desired_col = run_CNN(df, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                
            elif wm == 5:
                # Sequential & Spatial : LSTM-CNN
                m_name = 'LSTM-CNN'
                dict_out, X_test, Y_test = run_LSTM_CNN(df, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                

            # ----------------
            # Save all data to array 
            res_permod.append([ynum, m_name, fea, X_cols, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test, marquers_important, extra])
            # ----------------
            
        # Save data matrices to file per model result :
        file_name = "res_exp_%s_%s_%s_ynum%d_%s.pkl" % (exp, ax_val, ss_val, ynum, m_name)
        open_file = open(file_name, "wb")
        pickle.dump(res_permod, open_file)
        open_file.close()
            
    return

In [16]:
def mettez_dictout_dans_vars(dict_out, q):
    if q == 0:
        acc = dict_out['acc_train']
        prec = dict_out['prec_train']
        recall = dict_out['recall_train']
        roc_auc = dict_out['roc_auc_train']
    elif q == 1:
        acc = dict_out['acc_test']
        prec = dict_out['prec_test']
        recall = dict_out['recall_test']
        roc_auc = dict_out['roc_auc_test']
    elif q == 2:
        acc = dict_out['acc_dircalc']
        prec = dict_out['prec_dircalc']
        recall = dict_out['recall_dircalc']
        roc_auc = dict_out['rocauc_pp_dircalc']
    elif q == 3:
        acc = dict_out['acc_dircalc']
        prec = dict_out['prec_dircalc']
        recall = dict_out['recall_dircalc']
        roc_auc = dict_out['rocauc_pp_dircalc']
            
    return acc, prec, recall, roc_auc

In [50]:
def df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols):

    # Ensure that the X matrix size is correct
    # df_test2 : (dp_per_sample*n_values, feature)
    all_dp, cols = df_test2.shape

    # ----------------

    # Il faut change df_test2 à : (batch, timesteps, feature)
    needed_samps_class, counted_value, count_index, st, endd = count_classes(df_test2)

    tot = [endd[i]-st[i] for i in range(len(st))]
    val = min(tot)

    # Ensurez que X est le meme taille
    X = []
    Y = []
    for i in range(len(tot)):
        isamp = endd[i]-st[i]
        diff = isamp - val
        X.append(df_test2.iloc[st[i]:endd[i]-diff, X_cols].to_numpy())
        Y.append(df_test2.iloc[st[i]:endd[i]-diff, y_cols].to_numpy())

    # ----------------

    X = np.array(X)
    
    # Tensorflow says to use :
    batch, timesteps, feature = X.shape

    print('batch:' , batch)
    print('timesteps:' , timesteps)
    print('feature:' , feature)
    
    print('taille de X:' , X.shape)
    # X.shape =  (104570, 20, 1)   # batch, timesteps/sequence length, feature

    # ----------------

    Y = np.array(Y)
    batch, timesteps, n_outputs = Y.shape   # batch, timesteps, 1
    print('taille de Y:' , Y.shape)

    # ----------------

    # Split the X an y data into test and train
    seed = 0
    test_size = 0.25 # default

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = seed, test_size = test_size)

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    
    print('X_train:' , X_train.shape)
    print('Y_train:' , Y_train.shape)
    
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)
    print('X_test:' , X_test.shape)
    print('Y_test:' , Y_test.shape)
    
    # ----------------
    
    # Ensure that Y_train and Y_test are integers
    Y_test = Y_test.astype(int)
    Y_train = Y_train.astype(int)
    
    # ----------------
    
    # batch_train, timesteps_train, feature_train = X_train.shape
    # batch_test, timesteps_test, feature_test = X_test.shape
    
    # OU
    
    suf = ['train', 'test']
    noms = ['batch_', 'timesteps_', 'feature_']
    dictkeys = [j+i for i in suf for j in noms]
    #print('dictkeys: ', dictkeys)

    dictvals = []
    dictvals.append(list(X_train.shape))
    dictvals.append(list(X_test.shape))
    dictvals = np.ravel(dictvals)
    
    info = dict(zip(dictkeys, dictvals))
    info['n_outputs'] = n_outputs
    # ----------------
    
    return X_train, X_test, Y_train, Y_test, info

## CNN !!!

In [61]:
def initialize_CNN(df_test2, X_cols, y_cols, img_dim): 
    
    X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols)
    # X_train: (batch_train, timesteps_train, feature_train)
    # Y_train: (batch_train, timesteps_train, n_outputs)
    # X_test: (batch_test, timesteps_test, feature_train)
    # Y_test: (batch_test, timesteps_test, n_outputs)
    
    # ----------------
    
    # combinez timesteps et feature dans un image 
    X_train1 = np.reshape(X_train, (info['batch_train'], info['timesteps_train']*info['feature_train']))
    X_test1 = np.reshape(X_test, (info['batch_test'], info['timesteps_test']*info['feature_test']))
    # X_train: (batch_train, timesteps_train*feature_train)
    
    # ----------------
    
    X_train_img = mat2img(X_train1, img_dim, info['batch_train'])
    X_test_img = mat2img(X_test1, img_dim, info['batch_test'])
    
    # ----------------
    
    Y_train_1D =  [Y_train[i,0:1,0] for i in range(info['batch_train'])]
    Y_test_1D =  [Y_test[i,0:1,0] for i in range(info['batch_test'])]
    
    # ----------------
    
    # shape of X_train_img :  (batch_train, img_dim, ndes, 3)
    # shape of Y_train_1D :  (batch_train,)
    # shape of X_test_img :  (batch_test, img_dim, ndes, 3)
    # shape of Y_test_1D :  (batch_test,)
    
    return X_train_img, X_test_img, Y_train_1D, Y_test_1D, info
    
    

def mat2img(X, img_dim, batch):
    X_img = []
    for i in range(batch):
        mat = X[i,:]
        
        if i == 0:
            n = int(np.floor(np.sqrt(len(mat))))
        
        # fold into a square
        mat = np.reshape(mat, (n, n))
        
        # resize et convertir matrix au image : []
        img = Image.fromarray( mat , 'L')
        rgb_image = img.convert('RGB')
        
        # Resize image into a 64, 64, 3
        new_h, new_w = int(img_dim), int(img_dim)
        img3 = rgb_image.resize((new_w, new_h), Image.ANTIALIAS)
        w_resized, h_resized = img3.size[0], img3.size[1]
        
        # Convert image to an array
        image = np.array(img3)

        # Normalize image
        image = image/255
        
        X_img.append(image)
        
    
    return X_img

In [58]:
# 'Though for the 1D signal classification CNNs are also suitable, like they are implemented in the article 
# [49] for the seismic signal classification, while dealing with 2D objects CNNs can perform significantly 
# better results. Thus, firstly, we convert the 1D accelerometer signal into the 2D images via applying 
# CWT in order to extract signal features and, at the same time, to make it possible to implement 2D CNNs.'


# ----------------------------------------------
def MPCNN_arch(n_outputs, img_dim, ynum):
    
    # Typical architecture MPCNN architecture using alternating convolutional and max-pooling layers. 
    
    base_dimension = 32
    
    model = Sequential()  # initialize Sequential model
    
    # 32 channels, convolutional layer (kernel 5x5)
    model.add(Conv2D(base_dimension, (5,5), strides=(1,1), padding='same', input_shape=(img_dim, img_dim, 1)))
    
    # Max pooling layer, pool size 2x2, strides 2x2
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    # 64 channels, convolutional layer (kernel 5x5)
    model.add(Conv2D(base_dimension * 2, (5,5), strides=(1,1), padding='same'))

    # Max pooling layer, pool size 2x2, strides 2x2
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    # model.add(BatchNormalization(renorm=True))
    model.add(Flatten())
    
    model.add(Dense(1000, activation='relu'))

    initializer = tf.keras.initializers.HeUniform()
    # initializer = tf.keras.initializers.HeNormal()
    # initializer = tf.keras.initializers.GlorotUniform()
    if ynum == 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))
    
    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 1)
    # --------
    
    # Compile the model for training
    opt = keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    if ynum == 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    
    # model.summary()
    
    return model

In [59]:
# ----------------------------------------------
def dcgan_arch(n_outputs, img_dim, ynum):
    
    model = Sequential()
    model.add(Dense(hidden_dim * 4, activation=LeakyReLU(alpha=0.2), input_shape=img_dim*img_dim))
    model.add(Dense(hidden_dim * 2, activation=LeakyReLU(alpha=0.2)))
    model.add(Dense(hidden_dim, activation=LeakyReLU(alpha=0.2)))
    
    initializer = tf.keras.initializers.HeUniform()
    # initializer = tf.keras.initializers.HeNormal()
    # initializer = tf.keras.initializers.GlorotUniform()
    if ynum == 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))
    
    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 1)
    # --------
    
    # Compile the model for training
    opt = keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    if ynum == 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    
    # model.summary()
    
    return model

In [60]:
def encoderdecoder_arch(n_outputs, img_dim, ynum):
    
    mt = 0.9 # 0.15  défaut=0.99
    LyR_alpha = 0.2  # défaut=0.3
    
    base_dimension = 64          
    
    model = Sequential()
    # 1ère valeur (filters) : le nombre de tranches "(kernel_val,kernel_val)" qui composent l'image de sortie
    # 2eme valeur (kernel_size) : la taille de la carre/filtre que on glisse au dessous l'image 
    # 3eme valeur (stride): Le plus grande le stride valeur le plus petite l'image sortie : on prends z_dim/stride_num
    
    # --------
    # Entrée = (img_dim, img_dim, 1)
    model.add(Conv2D(base_dimension, (5,5), strides=(2,2), padding='same', input_shape=(img_dim, img_dim, 1)))
    print('model.output_shape :', model.output_shape)
    # Sortie = 
    # taille_sortie = (28 + 2*p - 5)/2 + 1

    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    # --------

    # --------
    # Entrée = 
    model.add(Conv2D(base_dimension * 2, (5,5), strides=(2,2), padding='same'))
    print('model.output_shape :', model.output_shape)
    # Sortie = 

    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    # --------

    # --------
    model.add(Flatten())

    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 4096)
    # --------
    
    initializer = tf.keras.initializers.HeUniform()
    # initializer = tf.keras.initializers.HeNormal()
    # initializer = tf.keras.initializers.GlorotUniform()
    if ynum == 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))
    
    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 1)
    # --------
    
    # Compile the model for training
    opt = keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    if ynum == 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    
    # model.summary()
    
    return model

In [ ]:
from PIL import Image
def mat2D_resize_mat2D_CNN(mat, img_dim):
    
    img = Image.fromarray( mat , 'L')
    
    # We preprocess your image to fit your algorithm.
    # img = Image.open(my_image)         # PIL: img is not in array form, it is a PIL.PngImagePlugin.PngImageFile 
    rgb_image = img.convert('RGB')

    # ----------------

    # Resize image into a 64, 64, 3
    new_h, new_w = int(img_dim), int(img_dim)
    img3 = rgb_image.resize((new_w, new_h), Image.ANTIALIAS)
    w_resized, h_resized = img3.size[0], img3.size[1]

    # ----------------

    # Convert image to an array
    image = np.array(img3)

    # ----------------

    # Transformer l'image de 3D à 2D
    # Convert image back to a 2D array
    mat_resized = np.mean(image, axis=2)
    
    # ----------------
    
    # Normalizer l'image entre 0 et 1
    mat_resized = mat_resized/255
    
    # OU
    
    # Normalize the images to [-1, 1]
    # mat_resized = (mat_resized - 127.5) / 127.5
    
    # ----------------
    
    useit = 0
    if useit == 1:
        # Threshold image
        thresh = 255/2
        row, col = mat_resized.shape
        mat_resized_th = np.ones((row, col))
        for i in range(row):
            for j in range(col):
                if mat_resized[i,j] > thresh:
                    mat_resized_th[i,j] = 0
        mat_resized = mat_resized_th
    
    return mat_resized

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization

tf.compat.v1.enable_eager_execution()  # This allows you to use placeholder in version 2.0 or higher
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

np.random.seed(1)

def run_CNN(df_test2, X_cols, y_cols, ynum):

    # ----------------
    img_dim = 64
    
    # Folding data into CNN image format:
    X_train_img, X_test_img, Y_train_1D, Y_test_1D, info  = initialize_CNN(df_test2, X_cols, y_cols, img_dim)
    batch = info['batch_train']
    timesteps_train = info['timesteps_train'] 
    feature = info['feature_train']
    n_outputs = info['n_outputs']
    
    # X_train et X_test pour dcgan
    for i in range()
    mat2D_resize_mat2D_CNN(mat, img_dim)
    
    
    # ----------------
    
    # Model architecture
    epochs = 100
    batch_size = 32
    
    tot = []
    tot_mod = []
    mod_type = ['mpcnn', 'dcgan', 'encdec'] # CNN model architecture type
    
    for i in range(3):
        if i == 0:
            model = MPCNN_arch(n_outputs, img_dim, ynum)
        elif i == 1:
            model = dcgan_arch(n_outputs, input_shape, ynum)
        elif i == 2:
            model = encoderdecoder_arch(n_outputs, img_dim, ynum)
    
        patience = 5 # Number of epochs with no improvement after which training will be stopped.
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')
        
        # -------------------------------
        
        if i == 1:
            
            # Reshape the training and test examples 
            train_x_flatten = X_train.reshape(X_train.shape[0], -1)   # The "-1" makes reshape flatten the remaining dimensions
            test_x_flatten = X_test.reshape(X_test.shape[0], -1)
            history = model.fit(X_train_, Y_train_1D, epochs=epochs, validation_data=(X_test_img, Y_test_1D), batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        else:
            history = model.fit(X_train_img, Y_train_1D, epochs=epochs, validation_data=(X_test_img, Y_test_1D), batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        
        
        history_df = pd.DataFrame(history.history)
        out = [history_df.iloc[:,i].mean() for i in range(len(history_df.columns))]
    
        tot.append(out)
        tot_mod.append(model)
    
    tot = np.array(tot)
    
    # -------------------------------
    
    a = np.argmax(tot[:,1])  # train
    b = np.argmax(tot[:,6])  # test
    suf = ['train', 'test']
    tr_noms = ['loss_', 'acc_', 'prec_', 'recall_', 'roc_auc_']

    list2 = [j+i for i in suf for j in tr_noms]
    list2

    dict_out = {}
    for i in range(len(list2)):
        if i < len(list2)/2:
            r = tot[a,i]
        else:
            r = tot[b,i]
        dict_out[list2[i]] = r


    # ajoutez au dictionaire
    dict_out['mod_train'] = mod_type[a]
    dict_out['mod_test'] = mod_type[b]

    # -------------------------------

    cnn2D_model_best = tot_mod[b]
    
    # -------------------------------
    
    return cnn2D_model_best, dict_out, X_test, Y_test

## LSTM !!

In [53]:
def initialize_LSTM(df_test2, X_cols, y_cols, return_sequences, stateless):
    
    X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols)
    
    # ----------------
    
    tf_train = X_Y_a_tfobj(X_train, Y_train, info['batch_train'], return_sequences, stateless)
    tf_test = X_Y_a_tfobj(X_test, Y_test, info['batch_test'], return_sequences, stateless)
    
    # side idea : if I give tf_test info['batch_train'] instead of info['batch_test'], could I run stateful=True
    # for all data (train and test)?
    # ----------------

    return tf_train, tf_test, X_test, Y_test, info



def X_Y_a_tfobj(X, Y, batch_size, return_sequences, stateless):
    
    if return_sequences == False:  # one output per batch of samples
        # Y shape : batch_size,      ie: Y.shape =  (104570,)
        temp = [Y[i,0:1,:] for i in range(Y.shape[0])]
        Y = np.array(temp)
        Y = np.reshape(Y, (batch_size,))
    elif return_sequences == True: # an output per each batch of samples
        # Y shape : batch_size, timesteps, n_output
        Y = np.array(Y)
    
    # ----------------
    
    tf_data = tf.data.Dataset.from_tensor_slices((X, Y))
    
    if stateless == True:  # sequence is non-causal
        buffer_size = 1000
        tf_data = tf_data.cache().shuffle(buffer_size).batch(batch_size)
    elif stateless == False:  # sequence is causal
        tf_data = tf_data.batch(batch_size)
    
    return tf_data

In [54]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

def run_LSTM(df_test2, X_cols, y_cols, ynum):

    
    return_sequences = False # True=return a prediction at every batch sample, (default) False=return one prediction at the end of the batch
    stateless = True # True=shuffle the batch samples/slices --samples are non-causal, False=do not shuffle slices---samples are causal
    
    # -------------------------------
    
    # Folding data into LSTM format:
    tf_train, tf_test, X_test, Y_test, info  = initialize_LSTM(df_test2, X_cols, y_cols, return_sequences, stateless)
    batch = info['batch_train']
    timesteps_train = info['timesteps_train'] 
    feature = info['feature_train']
    n_outputs = info['n_outputs']
    
    # -------------------------------

    # Model architecture
    epochs = 100
    batch_size = 32
    return_state = False # True=return a and c, (default) False=do not return hidden state (a) and cell state (c)
    stateful = False #  If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.
    
    tot = []
    tot_mod = []
    liste_de_vals = [20, 40, 60, 70, 80, 100] # number of dimensions for the hidden state of each LSTM cell
    
    for n_a in liste_de_vals:

        model = Sequential()

        if stateful == True:
            # Quand vous definez batch_input_shape, il faut que des entries de modele être le meme taille que le train dataset
            model.add(LSTM(n_a, input_shape=(timesteps_train, feature), batch_input_shape=(batch, timesteps_train, feature), return_sequences=return_sequences, return_state=return_state, stateful=stateful))
        elif stateful == False:
            model.add(LSTM(n_a, input_shape=(timesteps_train, feature), return_sequences=return_sequences, return_state=return_state, stateful=stateful))

        # Types of W initializer :
        initializer = tf.keras.initializers.HeUniform()
        
        if ynum == 2:
            model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
        else:
            model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))

        # Compile the model for training
        # opt = keras.optimizers.Adam()
        opt = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
        # opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

        # Si vous utilisez softmax activation, la taille de sortie est plus grand que deux donc il faut categorical_crossentropy
        if ynum == 2:
            model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
        else:
            model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'

        # -------------------------------
        
        # 50 est trop, 20 est insuffisant 
        patience = 1 # Number of epochs with no improvement after which training will be stopped.
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')
        
        # -------------------------------

        if stateful == False:
            history = model.fit(tf_train, epochs=epochs, validation_data=tf_test, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
            # OU
            # history = model.fit(tf_train, epochs=epochs, batch_size=batch_size, verbose=2)
            # score = model.evaluate(tf_test, verbose=2)
            # test_acc.append(score[1])
            # test_loss.append(score[0])
        elif stateful == True:
            # On peut faire tf_train parce que le taille est fixer at batch_input_shape
            history = model.fit(tf_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
    
        
        history_df = pd.DataFrame(history.history)
        out = [history_df.iloc[:,i].mean() for i in range(len(history_df.columns))]
        
        tot.append(out)
        tot_mod.append(model)
    
    tot = np.array(tot)
    
    # -------------------------------
    
    a = np.argmax(tot[:,1])  # train
    b = np.argmax(tot[:,6])  # test
    suf = ['train', 'test']
    tr_noms = ['loss_', 'acc_', 'prec_', 'recall_', 'roc_auc_']

    list2 = [j+i for i in suf for j in tr_noms]
    list2

    dict_out = {}
    for i in range(len(list2)):
        if i < len(list2)/2:
            r = tot[a,i]
        else:
            r = tot[b,i]
        dict_out[list2[i]] = r


    # ajoutez au dictionaire
    dict_out['delay_train'] = liste_de_vals[a]
    dict_out['delay_test'] = liste_de_vals[b]

    # -------------------------------

    lstm_model_best = tot_mod[b]
    
    # -------------------------------
    
    return lstm_model_best, dict_out, X_test, Y_test

# Batch Run

In [20]:
def is_empty(vec):
    
    #if not any(vec):
    
    # OR
    
    #if len(vec) < 1:  # OR
    
    vec = np.array(vec)
    if vec.shape[0] == 0:
        # print('yes, the array is empty')
        out = True
    else:
        # print('no, the array is not empty')
        out = False
        
    return out

In [21]:
def make_a_properlist(vec):
    
    out = []
    for i in range(len(vec)):
        out = out + [np.ravel(vec[i])]
        
    if is_empty(out) == False:
        vecout = np.concatenate(out).ravel().tolist()
    else:
        vecout = list(np.ravel(out))
    
    return vecout

In [22]:
def run_main_script(exp, ax_val, ss_val):
    
    # ----------------
    
    df = get_df(ax_val, ss_val)

    feat0, t_feat0, y1_feat0, y2_feat0, y3_feat0 = create_labels_and_initial_feature(df) 
    # Elasped time for feature processing :  715.9615476131439

    del df

    # ----------------

    if exists("df_feat.pkl") == False:
        print('Creation des marquers')
        df_feat = feature_creation_preprocessing(feat0, t_feat0)
        del feat0, t_feat0
        save_dat_pickle(df_feat, file_name="df_feat.pkl")
    else:
        print('Load des marquers')
        df_feat = load_dat_pickle(file_name="df_feat.pkl")
        
    df_org = df_feat
    
    del df_feat
    # ----------------

    col_list = ['num', 'pos', 'vel', 'acc', 'pos_nc', 'vel_nc', 'acc_nc', 'pos_sl0', 'pos_sl1', 'pos_sl2', 'pos_sl3', 'pos_sl4', 'vel_sl0', 'vel_sl1', 'vel_sl2', 'vel_sl3', 'vel_sl4', 'acc_sl0', 'acc_sl1', 'acc_sl2', 'acc_sl3', 'acc_sl4', 'pos_spec', 'vel_spec', 'acc_spec', 'pos_cwt', 'vel_cwt', 'acc_cwt']
    df_org = pandas_rename_columns(df_org, col_list)
    df_org = df_org.reset_index(drop=True)  # reset index : delete the old index column
    # Gardez df_org
    
    # ----------------

    y_alllabel = [y1_feat0, y2_feat0, y3_feat0]
    
    # ----------------
    
    # Loop over the different y labels
    for ynum in range(1):    #range(len(y_alllabel)):
        print('ynum : ', ynum)
        
        # ----------------
        
        # Select y
        y_label = y_alllabel[ynum]
        
        # ----------------

        # Ajoutez le column de label à la fin
        y_pd = pd.Series(make_a_properlist(y_label))

        # ----------------

        df_feat = pd.concat([df_org, y_pd], axis=1)
        df_feat = df_feat.rename({0: 'y'}, axis=1)

        # ----------------

        # Balancez des class/labels: pad
        # Pad data 2 Make Classes Equivalent
        df_test2 = pad_data_2makeclasses_equivalent(df_feat)
        del df_feat
        
        # ----------------
        
        # classification for 3 partitions of feature data per 8 classifiers
        # classify(df_test2, ynum)
        
        # del df_test2

    return df_test2

In [23]:
import warnings
warnings.filterwarnings('ignore')

exp = 'rot'
ax_val = 'all'
ss_val = 'all'
df_test2 = run_main_script(exp, ax_val, ss_val)

Confirmation : exp= rot , ax_val= all , ss_val= all
Load des marquers
ynum :  0
Are there nan valeus in the data :  False
Before sorting counted_value :  [294, 96]
Before sorting count_index :  [0, 1]
After sorting counted_value :  [294, 96]
After sorting count_index :  [0, 1]
needed_samps_class :  [  0 198]
shape of dataframe before padding :  (183690, 29)
shape of dataframe to add to original dataframe:  (93060, 29)
shape of padded dataframe (original + toadd) :  (276750, 29)
Before sorting counted_value :  [294, 294]
Before sorting count_index :  [1, 0]
After sorting counted_value :  [294, 294]
After sorting count_index :  [0, 1]
needed_samps_class :  [0 0]


In [ ]:
out = load_dat_pickle(file_name="res_exp_rot_all_all_ynum0_LSTM_fea3.pkl")
dff = pd.DataFrame(out)
dff

# Individual tests

In [35]:
fea = 0

if fea == 0:
    # 1) All features
    X_cols = list(np.arange(1, df_test2.shape[1]-1, 1))
    y_cols = [df_test2.shape[1]-1]
elif fea == 1:
    # 2) first 3 from permutation_importance
    X_cols = [list(marquers_important.items())[oo][0] for oo in range(3)]
    y_cols = [df_test2.shape[1]-1]
elif fea == 2:
    # 3) first 2 from permutation_importance
    X_cols = [list(marquers_important.items())[oo][0] for oo in range(2)]
    y_cols = [df_test2.shape[1]-1]
elif fea == 3:
    # 4) first feature from permutation_importance
    X_cols = [list(marquers_important.items())[0][0]]
    y_cols = [df_test2.shape[1]-1] 


m_name = 'NuSVC'
X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2)

# Determine if classes are binary or multiclass:
class_len = len(np.unique(Y_train))
print('class_len: ', class_len)


batch_size = 16
model, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test = binary_svm_NuSVC_batch(X_train, X_test, Y_train_1D, Y_test_1D, batch_size)
extra = np.nan
# ----------------
# Permutation importance of features : probe which features are most predictive
if fea == 0:
    from sklearn.inspection import permutation_importance
    r = permutation_importance(model, X_test, Y_test_1D, n_repeats=10, random_state=0, scoring='accuracy')
    vals = dict(zip(np.arange(len(r.importances_mean)), r.importances_mean))
    marquers_important = sort_dict_by_value(vals, reverse = True) # Sort the columns from largest to smallest mae
# ----------------

Before sorting counted_value :  [294, 294]
Before sorting count_index :  [1, 0]
After sorting counted_value :  [294, 294]
After sorting count_index :  [0, 1]
needed_samps_class :  [0 0]
batch: 588
timesteps: 469
feature: 27
taille de X: (588, 469, 27)
taille de Y: (588, 469, 1)
X_train: (441, 469, 27)
Y_train: (441, 469, 1)
X_test: (147, 469, 27)
Y_test: (147, 469, 1)
class_len:  2


In [44]:
def evaluation_methods_binary_class(model, X, Y_1D, Y_1D_predict, Y_bin_pp, Y_1D_score):
    
    # ----------------------------

    # 4) Direct calculation of metrics
    from sklearn import metrics

    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

    # labels = array-like, default=None

    # pos_label = str or int, default=1 The class to report if average='binary' and the data is binary. 
    # If the data are multiclass or multilabel, this will be ignored

    # average = ['binary', 'micro', 'macro', 'weighted', 'samples', None]
    # This parameter is required for multiclass/multilabel targets. 
    # None : the scores for each class are returned
    # 'binary'  : Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.
    # 'micro' : Calculate metrics globally by counting the total true positives, false negatives and false positives.
    # 'macro' : Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
    # 'weighted' : Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.
    # 'samples' : Calculate metrics for each instance, and find their average (only meaningful for multilabel classification where this differs from accuracy_score).
    average = 'micro'

    # sample_weight : array-like of shape (n_samples,), default=None
    acc_dircalc = metrics.accuracy_score(Y_1D, Y_1D_predict)
    prec_dircalc = metrics.precision_score(Y_1D, Y_1D_predict, average=average)
    recall_dircalc = metrics.recall_score(Y_1D, Y_1D_predict, average=average)
    f1_dircalc = metrics.f1_score(Y_1D, Y_1D_predict, average=average)
    
    # Y_bin_pp is size [n_samples, n_classes=2]
    # Take the column of Y_bin_pp for the class of Y_1D, because both vectors need to be [n_samples, 1]
    Y_1D_pp = []
    for q in range(len(Y_1D)):
        desrow = Y_bin_pp[q]
        Y_1D_pp.append(desrow[int(Y_1D[q])])
    Y_1D_pp = np.ravel(Y_1D_pp)
    
    Y_1D_pp = np.array(Y_1D_pp)
    Y_1D = np.array(Y_1D)
    
    rocauc_pp_dircalc = metrics.roc_auc_score(Y_1D, Y_1D_pp, average=average) # prediction probability
    
    # ----------------------------
    
    value_pack = {}
    
    var_list = ['acc_dircalc', 'prec_dircalc', 'recall_dircalc', 'f1_dircalc', 'rocauc_pp_dircalc']
    var_list_num = [acc_dircalc, prec_dircalc, recall_dircalc, f1_dircalc, rocauc_pp_dircalc]
    
    for q in range(len(var_list)):
        value_pack['%s' % (var_list[q])] = var_list_num[q]
    
    return value_pack

In [45]:
from sklearn import svm
import os

def binary_svm_NuSVC_batch(X_train, X_test, Y_train_1D, Y_test_1D, batch_size):

    n = int(np.ceil(info['batch_train']/batch_size))
    print('n: ', n)
    
    results = []
    
    for i in range(1):  # range(n)
        st = i*batch_size
        endd = st+batch_size

        # Stack all data over batches
        X_train_batch = np.reshape(X_train[st:endd,:,:], (batch_size*info['timesteps_train'], info['feature_train']))
        Y_train_1D_batch = np.reshape(Y_train[st:endd,:,:], (batch_size*info['timesteps_train'], info['n_outputs']))
        X_test_batch = np.reshape(X_test[st:endd,:,:], (batch_size*info['timesteps_test'], info['feature_test']))
        Y_test_1D_batch = np.reshape(Y_test[st:endd,:,:], (batch_size*info['timesteps_test'], info['n_outputs']))

        print('shape of X_train_batch : ', X_train_batch.shape)
        print('shape of Y_train_1D_batch : ', Y_train_1D_batch.shape)
        print('shape of X_test_batch : ', X_test_batch.shape)
        print('shape of Y_test_1D_batch : ', Y_test_1D_batch.shape)
        
        model = svm.NuSVC(decision_function_shape='ovo', probability=True, max_iter=-1)  # “one-versus-one” : binary ONLY, Y_train_1D, same implementation as libsvm (uses 1/lambda instead of C in cost function)
        model.fit(X_train_batch, Y_train_1D_batch)
        
        # OU
        
        # from sklearn.pipeline import make_pipeline
        # from sklearn.preprocessing import StandardScaler
        # from sklearn.svm import NuSVC
        # model = make_pipeline(StandardScaler(), NuSVC())
        # model.fit(X_train_batch, Y_train_1D_batch)
        
        # ----------------------------

        Y_train_1D_predict = model.predict(X_train_batch)
        Y_test_1D_predict = model.predict(X_test_batch)

        # The prediction probability of each class : is size [n_samples, n_classes]
        Y_train_bin_pp = model.predict_proba(X_train_batch) 
        Y_test_bin_pp = model.predict_proba(X_test_batch)

        Y_train_bin_pp = np.array(Y_train_bin_pp)
        print('shape of Y_train_bin_pp : ', Y_train_bin_pp.shape)
        Y_test_bin_pp = np.array(Y_test_bin_pp)
        print('shape of Y_test_bin_pp : ', Y_test_bin_pp.shape)

        # How confidently each value predicted for x_test by the classifier is Positive ( large-magnitude Positive value ) or Negative ( large-magnitude Negative value)
        Y_train_1D_score = model.decision_function(X_train_batch)  # size is [n_samples, 1]
        Y_test_1D_score = model.decision_function(X_test_batch)

        Y_train_1D_score = np.array(Y_train_1D_score)
        print('shape of Y_train_1D_score : ', Y_train_1D_score.shape)
        Y_test_1D_score = np.array(Y_test_1D_score)
        print('shape of Y_test_1D_score : ', Y_test_1D_score.shape)

        # ----------------------------

        value_pack_train = evaluation_methods_binary_class(model, X_train_batch, Y_train_1D_batch, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
        value_pack_test = evaluation_methods_binary_class(model, X_test_batch, Y_test_1D_batch, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)

        # ----------------------------
        
        acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(value_pack_train, q=2)
        acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(value_pack_test, q=3) 
        
        results.append(i, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test)
        
        # Save model to file
        file_name = "model_%s.pkl" % (i)
        save_dat_pickle(model, file_name=file_name)
        
        # Delete model
        del model
        del Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score
        
    
    # Evaluatez quel modeles est mieux
    best = np.argmax(results[:,6])
    acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test = map(float, results[:,6])
    
    # Load best model only for permutation importance
    file_name = "model_%s.pkl" % (best)
    model = load_dat_pickle(file_name=file_name)
    
    # Delete all .pkl files
    del results
    rm_list = [i for i in range(n) if i != best]
    for i in rm_list:
        os.remove("model_%s.pkl" % (i))
    
    return model, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test